In [8]:
import os
import pandas as pd
import yaml

In [7]:
#!pip install pyyaml

  Obtaining dependency information for pyyaml from https://files.pythonhosted.org/packages/b3/34/65bb4b2d7908044963ebf614fe0fdb080773fc7030d7e39c8d3eddcd4257/PyYAML-6.0.1-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/144.7 kB ? eta -:--:--
   -- ------------------------------------- 10.2/144.7 kB ? eta -:--:--
   -------- ------------------------------ 30.7/144.7 kB 660.6 kB/s eta 0:00:01
   ---------------------------------------- 144.7/144.7 kB 1.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\de129790\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [19]:
def read_yaml_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        data = yaml.safe_load(file)
        if data is None:
            print(f"Warning: {filepath} is empty.")
            return {}  # Return an empty dictionary for empty files
    return data

In [13]:
def yaml_to_dataframe(yaml_dict):
    # Check for the 'hypotheses' key
    if 'hypotheses' not in yaml_dict:
        raise ValueError(f"'hypotheses' key not found in the YAML data")
    
    # Flatten the hypotheses section
    hypotheses_data = []
    for idx, hypothesis in yaml_dict['hypotheses'].items():
        hypothesis['hypothesis_id'] = idx
        hypotheses_data.append(hypothesis)
    
    hypotheses_df = pd.DataFrame(hypotheses_data)
    
    # Repeat other data for each hypothesis
    for key, value in yaml_dict.items():
        if key != 'hypotheses':
            hypotheses_df[key] = value
    
    return hypotheses_df


In [21]:
directory = 'chatGPT_results'
all_dataframes = []

# Ensure the directory exists
if not os.path.exists(directory):
    print(f"The directory {directory} does not exist.")
else:
    # Process each YAML file in the specified directory
    for filename in os.listdir(directory):
        if filename.endswith('.yaml') or filename.endswith('.yml'):
            filepath = os.path.join(directory, filename)
            yaml_dict = read_yaml_file(filepath)
            if yaml_dict:  # Only process non-empty dictionaries
                try:
                    df = yaml_to_dataframe(yaml_dict)
                    all_dataframes.append(df)
                except ValueError as e:
                    print(f"Error processing {filepath}: {e}")

    # Concatenate all DataFrames
    final_dataframe = pd.concat(all_dataframes, ignore_index=True)

    # Optionally, save the final DataFrame to a CSV file
    # final_dataframe.to_csv('output.csv', index=False)

    # Display the final DataFrame
    final_dataframe


AttributeError: 'NoneType' object has no attribute 'items'